In [ ]:
!pip install keybert

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from keybert import KeyBERT
from collections import Counter

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
data = pd.read_csv("Maritime Accidents Reports - Overall Data.csv")

In [ ]:
data

,Summary,Safety Issues,Location,Title,Date Occurred,Unnamed: 5,Decision Events,Skill Based Events,Perception Events,Routine Violation,...,Adverse Physiological,Medical / Mental,Individual Perception,Inadequate Supervision,Failure to Correct a Known Problem,Planned Inappropriate Operations,Supervisory Violations,Organizational Culture / Climate,Organizational Processes,Resource Management
0,"At 1400 on 5 July 2022, the UK registered roll...",routinely passing too close to land\n\nfatigue...,"east coast of Swona Island, Pentland Firth, Sc...",Grounding of roll-on/roll-off passenger ferry ...,5 July 2022,NaN,0,0,0,1,...,1,0,0,0,0,0,0,1,1,0
1,"On 2 October 2022, the mono-hulled offshore po...",Steering control was lost when a drive shaft f...,Loss of control of powerboat Awesome with loss...,"near Little Thatch Island, British Virgin Islands",22 October 2022,NaN,1,0,0,1,...,0,1,0,0,1,1,0,0,0,0
2,"On 7 October 2022, a deckhand from the UK regi...",The unwritten on board process for shooting ne...,approximately 150 nautical miles west of Ireland,Fall overboard from fishing vessel Eder Sands ...,7 October 2022,NaN,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
3,"On the morning of 22 August 2020, the commerci...",The skipper did not see the buoy in sufficient...,"Southampton Water, England",Heavy contact between the high-speed passenger...,22 August 2020,NaN,1,0,1,0,...,1,1,0,0,0,1,1,1,1,1
4,"On 15 November 2019, an apprentice engineer di...",the fire-extinguishing aerosol was hazardous t...,"Alongside North Pier at Newlyn Harbour, Cornwa...",Accidental discharge of condensed aerosol fire...,15 November 2019,NaN,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,About 0048 Alaska Standard Time on February 16...,​​The probable cause of the grounding of the S...,AK,Grounding of Commercial Fishing Vessel Savanna...,2/15/2015,NaN,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
705,"About 1510 local time on January 22, 2015, the...",​​The probable cause of the flooding and event...,"Oahu, HI",Marine Accident Brief: Sinking of Towing Vesse...,1/22/2015,NaN,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
706,"​​About 1430 on September 23, 2014 t​he fishin...",The probable cause of the loss of the fishing ...,"Southwest Pass, LA",Capsizing and Sinking of the Fishing Vessel Ch...,9/23/2014,NaN,0,1,0,1,...,0,0,0,0,0,0,1,0,1,0
707,"​​On March 12, 2013, at 1745 local time, the t...",​​We determined that the probable cause of the...,"Bayou Perot, LA",Fire on Board Towing Vessel Shanon E. Settoon,3/12/2013,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
data = data.drop(["Unnamed: 5"], axis=1)
data = data.sample(frac=1, ignore_index=True)

In [ ]:
def clean_text(text):
  text = text.lower()
  text = re.sub('\n', '', text)
  return text

In [ ]:
data["Summary"] = data["Summary"].apply(clean_text)
data["Safety Issues"] = data["Safety Issues"].apply(clean_text)

In [ ]:
data["Overall Info"] = data["Summary"] + " " + data["Safety Issues"]

In [ ]:
data.duplicated().sum()

0

In [ ]:
cols = {'Medical / Mental' : 'Medical(or)Mental', 'Organizational Culture / Climate' : 'Organizational Culture(or)Climate'}

In [ ]:
data = data.rename(cols, axis=1)

In [ ]:
data_labels = data.drop(['Summary', 'Safety Issues', 'Location', 'Title', 'Date Occurred', 'Overall Info'], axis=1)
categories = list(data_labels.columns.values)

In [ ]:
model2 = PorterStemmer()
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
import nltk
from nltk.corpus import stopwords
import string

def message_transform(text):
    text = text.lower()
    text = nltk.word_tokenize(text)

    req_words = []
    for i in text:
      if i != "summary" and i != "safety issues" and i != "probable" and i != "cause" and i != "vessel" and i != "vessels" and i != "crew":
        if i.isalnum():
            req_words.append(i)

    req_words1 = []
    for i in req_words:
        if i not in stopwords.words("english") and i not in string.punctuation:
                req_words1.append(i)

    # req_words2 = []
    # for i in req_words1:
    #     req_words2.append(model2.stem(i))

    return " ".join(req_words1)

In [ ]:
message_transform("Hello World! and nice to meeting you**20")

'hello world nice meeting 20'

In [ ]:
data["Overall Info"] = data["Overall Info"].apply(message_transform)
data["Safety Issues"] = data["Safety Issues"].apply(message_transform)

In [ ]:
for category in categories:
    word_cloud = WordCloud(height=500, width=500, min_font_size=10, background_color="black")
    spam_word_cloud = word_cloud.generate(data[data[category] == 1]["Safety Issues"].str.cat(sep=" "))

    plt.figure(figsize=(14, 7))
    plt.imshow(spam_word_cloud, interpolation='bilinear')
    plt.axis("off")
    plt.title("Label " + category, fontsize=20)
    plt.savefig(f"{category}_safetyIssue_wordcloud.png", format="png", bbox_inches="tight")
    plt.show()

In [ ]:
from collections import Counter
from keybert import KeyBERT

sentence_transformers_models = ["all-mpnet-base-v2", "multi-qa-mpnet-base-dot-v1", "all-distilroberta-v1", "all-MiniLM-L12-v2", "multi-qa-distilbert-cos-v1"]

for m in sentence_transformers_models:
    kw_model = KeyBERT(model = m)

    with open(f"top_keywords_{m}.txt", 'w') as file:
        for category in categories:
            filtered_texts = data[data[category] == 1]["Overall Info"]
            keywords_list = []

            for text in filtered_texts:
                keywords = kw_model.extract_keywords(text, top_n=10)
                keywords_list.extend([kw[0] for kw in keywords])

            keyword_freq = Counter(keywords_list)
            top_keywords = keyword_freq.most_common(10)

            file.write(f"Top 10 Keywords for label '{category}':\n")
            for keyword, freq in top_keywords:
                file.write(f"{keyword}: {freq} occurrences\n")
            file.write("\n")

In [ ]:
!pip install keybert[flair]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.3 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=92de0c6b3e297f9c58a8f98c659b51f0e505f54f413fea0375529ce0ba29c78f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a37063

In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings

# transformers_models = ["roberta-base", "bert-base-uncased", "distilbert-base-uncased"]
transformers_models = ["distilbert-base-uncased"]

for model in transformers_models:
    m = TransformerDocumentEmbeddings(model)
    kw_model = KeyBERT(model = m)
    with open(f"top_keywords_{model}.txt", 'w') as file:
        for category in categories:
            filtered_texts = data[data[category] == 1]["Overall Info"]
            keywords_list = []

            for text in filtered_texts:
                keywords = kw_model.extract_keywords(text, top_n=10)
                keywords_list.extend([kw[0] for kw in keywords])

            keyword_freq = Counter(keywords_list)
            top_keywords = keyword_freq.most_common(10)

            file.write(f"Top 10 Keywords for label '{category}':\n")
            for keyword, freq in top_keywords:
                file.write(f"{keyword}: {freq} occurrences\n")
            file.write("\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
!pip install spacy pytextrank
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import pytextrank
import pandas as pd
import numpy as np
from collections import defaultdict

/usr/local/lib/python3.10/dist-packages


In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")

In [ ]:
label_keywords = defaultdict(list)

In [ ]:
for index, row in data.iterrows():
    text = row['Overall Info']
    doc = nlp(text)

    keywords = [phrase.text for phrase in doc._.phrases[:10]]

    for label in categories:
        if row[label] == 1:
            label_keywords[label].extend(keywords)

In [ ]:
for label, keywords in label_keywords.items():
    keywords = list(set(keywords))
    with open(f"{label}_keywords.txt", "w") as f:
        f.write("\n".join(keywords))